In [1]:
import findspark
findspark.init()
import pyspark

Steps can be found on https://docs.delta.io/latest/quick-start.html

In [2]:
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [3]:
from delta.tables import *

# Create a table

To create a Delta table, write a DataFrame out in the delta format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

These operations create a new Delta table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta table, see Create a table and Write to a table.

# Read data

You read data in your Delta table by specifying the path to the files: "/tmp/delta-table":

In [5]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



# Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table:


In [6]:
# Overwrite
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")


If you read this table again, you should see only the values 5-9 you have added because you overwrote the previous data.m

# Conditional update without overwrite

Delta Lake provides programmatic APIs to conditional update, delete, and merge (upsert) data into tables. Here are a few examples.

In [7]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()


+---+
| id|
+---+
|  2|
| 15|
| 16|
|  7|
|  3|
| 19|
|  6|
|  0|
|  5|
|  4|
|  9|
| 17|
| 12|
| 10|
|  1|
| 11|
| 18|
| 14|
| 13|
|  8|
+---+



You should see that some of the existing rows have been updated and new rows have been inserted.

# Read older versions of data using time travel

You can query previous snapshots of your Delta table by using time travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.

In [8]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



You should see the first set of data, from before you overwrote it. Time travel takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see Query an older snapshot of a table (time travel).

# Write a stream of data to a table

You can also write to a Delta table using Structured Streaming. The Delta Lake transaction log guarantees exactly-once processing, even when there are other streams or batch queries running concurrently against the table. By default, streams run in append mode, which adds new records to the table:

In [9]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

# Read a stream of changes from a table

While the stream is writing to the Delta table, you can also read from that table as streaming source. For example, you can start another streaming query that prints all the changes made to the Delta table. You can specify which version Structured Streaming should start from by providing the startingVersion or startingTimestamp option to get changes from that point onwards. See Structured Streaming for details.

stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()

# Writing SQL

You can also write SQL statements with the `USE DELTA` argument, like creating a table called `events`:

In [10]:
spark.sql('CREATE TABLE events(date DATE,  eventId STRING,  eventType STRING,  data STRING) USING DELTA')

DataFrame[]

And read from the newly created table:

In [11]:
spark.sql('SELECT * FROM events')

DataFrame[date: date, eventId: string, eventType: string, data: string]

# Loading data from the internet

In [16]:
!wget -q -O covid-01-01-2021.csv "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2021.csv"

In [17]:
!wget -q -O covid-01-25-2020.csv "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv"

## Alternative

In [18]:
from pyspark import SparkFiles

In [19]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv")

In [43]:
covid_df = spark.read.csv(SparkFiles.get("01-25-2020.csv"), header=True)

In [24]:
covid_df.show()

+--------------+--------------+-------------+---------+------+---------+
|Province/State|Country/Region|  Last Update|Confirmed|Deaths|Recovered|
+--------------+--------------+-------------+---------+------+---------+
|         Hubei|Mainland China|1/25/20 17:00|      761|    40|       32|
|     Guangdong|Mainland China|1/25/20 17:00|       78|  null|        2|
|      Zhejiang|Mainland China|1/25/20 17:00|       62|  null|        1|
|     Chongqing|Mainland China|1/25/20 17:00|       57|  null|     null|
|         Hunan|Mainland China|1/25/20 17:00|       43|  null|     null|
|       Beijing|Mainland China|1/25/20 17:00|       41|  null|        2|
|         Anhui|Mainland China|1/25/20 17:00|       39|  null|     null|
|      Shanghai|Mainland China|1/25/20 17:00|       33|  null|        1|
|         Henan|Mainland China|1/25/20 17:00|       32|  null|     null|
|       Sichuan|Mainland China|1/25/20 17:00|       28|  null|     null|
|      Shandong|Mainland China|1/25/20 17:00|      

## Loading JSON

In [46]:
!wget -q -O bomen.json "https://ckan.dataplatform.nl/api/action/datastore_search?resource_id=2d6893b4-d56d-4865-b6cc-0bda42e547f5"

In [47]:
import json

In [68]:
with open('bomen.json') as f:
  data = json.load(f)

In [73]:
records = data['result']['records']

In [111]:
json.dumps(records)

'[{"_id": 1, "Nederlandse naam": "Kegelvormige haagbeuk", "Wetenschappelijke naam": "Carpinus betulus \'Fastigiata\'", "Plantjaar": 1980, "Leeftijd": 40, "Eigenaar": "Gemeentelijk", "Buurt": "Rubicondreef e.o.                      ", "Wijk": "Overvecht", "Boomnummer": 1847669, "X": 136618.973, "Y": 458309.16000000003, "Lat": 5.118863998850161, "Long": 52.11270618246065, "Exportdatum": 20200415}, {"_id": 2, "Nederlandse naam": "Gewone plataan", "Wetenschappelijke naam": "Platanus x hispanica", "Plantjaar": 1935, "Leeftijd": 85, "Eigenaar": "Gemeentelijk", "Buurt": "Mari\\u00ebndaalstraat e.o.                  ", "Wijk": "Noordwest", "Boomnummer": 1890560, "X": 135163.508, "Y": 457391.218, "Lat": 5.097669760253711, "Long": 52.104405555042945, "Exportdatum": 20200415}, {"_id": 3, "Nederlandse naam": "Gewone plataan", "Wetenschappelijke naam": "Platanus x hispanica", "Plantjaar": 1927, "Leeftijd": 93, "Eigenaar": "Gemeentelijk", "Buurt": "J.P. Coenstraat-west                   ", "Wijk": "

In [93]:
rdd = spark.sparkContext.parallelize([records_j])

In [95]:
trees_df = spark.read.json(rdd)

In [110]:
trees_df.show()

+----------+--------------------+------------+-----------+------------------+--------+------------------+--------------------+---------+----------------------+------------------+----------+------------------+---+
|Boomnummer|               Buurt|    Eigenaar|Exportdatum|               Lat|Leeftijd|              Long|    Nederlandse naam|Plantjaar|Wetenschappelijke naam|              Wijk|         X|                 Y|_id|
+----------+--------------------+------------+-----------+------------------+--------+------------------+--------------------+---------+----------------------+------------------+----------+------------------+---+
|   1847669|Rubicondreef e.o....|Gemeentelijk|   20200415| 5.118863998850161|      40| 52.11270618246065|Kegelvormige haag...|     1980|  Carpinus betulus ...|         Overvecht|136618.973|458309.16000000003|  1|
|   1890560|Mariëndaalstraat ...|Gemeentelijk|   20200415| 5.097669760253711|      85|52.104405555042945|      Gewone plataan|     1935|  Platanus x

In [107]:
pd.read_json(records_j)

,_id,Nederlandse naam,Wetenschappelijke naam,Plantjaar,Leeftijd,Eigenaar,Buurt,Wijk,Boomnummer,X,Y,Lat,Long,Exportdatum
0,1,Kegelvormige haagbeuk,Carpinus betulus 'Fastigiata',1980,40,Gemeentelijk,Rubicondreef e.o.,Overvecht,1847669,136618.973000,458309.160000,5.118864,52.112706,20200415
1,2,Gewone plataan,Platanus x hispanica,1935,85,Gemeentelijk,Mariëndaalstraat e.o.,Noordwest,1890560,135163.508000,457391.218000,5.097670,52.104406,20200415
2,3,Gewone plataan,Platanus x hispanica,1927,93,Gemeentelijk,J.P. Coenstraat-west,West,1847994,134879.564000,455664.948000,5.093627,52.088880,20200415
3,4,Oosterse plataan,Platanus orientalis,2014,6,Gemeentelijk,Schaepmanplein e.o.,Noordwest,1850914,134392.715000,457649.375000,5.086404,52.106698,20200415
4,5,Gewone plataan,Platanus x hispanica,1979,41,Gemeentelijk,"Zamenhofdreef-west, GWC",Overvecht,1856228,136225.989000,458442.259000,5.113120,52.113889,20200415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Gewone esdoorn,Acer pseudoplatanus,1985,35,Gemeentelijk,Brennerbaan e.o.,Zuid,1856388,137716.944000,453179.260000,5.135153,52.066634,20200415
96,96,Gewone plataan,Platanus x hispanica,1925,95,Gemeentelijk,Hengeveldstraat e.o.,Noordoost,1856393,137520.082000,456961.173000,5.132088,52.100620,20200415
97,97,Gewone plataan,Platanus x hispanica,2015,5,Gemeentelijk,"Hoog Catharijne, Leidseveer, NS",Binnenstad,1850385,136012.929458,456282.153817,5.110130,52.094467,20200415
98,98,Hartbladige els,Alnus cordata,1996,24,Gemeentelijk,De Tol,Vleuten - De Meern,1856533,130163.721000,457948.592000,5.024653,52.109213,20200415


In [109]:
ks.from_pandas(pd.read_json(records_j))

,_id,Nederlandse naam,Wetenschappelijke naam,Plantjaar,Leeftijd,Eigenaar,Buurt,Wijk,Boomnummer,X,Y,Lat,Long,Exportdatum
0,1,Kegelvormige haagbeuk,Carpinus betulus 'Fastigiata',1980,40,Gemeentelijk,Rubicondreef e.o.,Overvecht,1847669,136618.973000,458309.160000,5.118864,52.112706,20200415
1,2,Gewone plataan,Platanus x hispanica,1935,85,Gemeentelijk,Mariëndaalstraat e.o.,Noordwest,1890560,135163.508000,457391.218000,5.097670,52.104406,20200415
2,3,Gewone plataan,Platanus x hispanica,1927,93,Gemeentelijk,J.P. Coenstraat-west,West,1847994,134879.564000,455664.948000,5.093627,52.088880,20200415
3,4,Oosterse plataan,Platanus orientalis,2014,6,Gemeentelijk,Schaepmanplein e.o.,Noordwest,1850914,134392.715000,457649.375000,5.086404,52.106698,20200415
4,5,Gewone plataan,Platanus x hispanica,1979,41,Gemeentelijk,"Zamenhofdreef-west, GWC",Overvecht,1856228,136225.989000,458442.259000,5.113120,52.113889,20200415
5,6,Schietwilg,Salix alba,2009,11,Gemeentelijk,"Zonnebaan, Sterrebaan e.o.",West,1851042,131441.238000,459003.312000,5.043229,52.118749,20200415
6,7,Gewone plataan,Platanus x hispanica,2012,8,Gemeentelijk,Langerak-Oost,Leidsche Rijn,1853194,133583.504000,455305.424000,5.074739,52.085600,20200415
7,8,Witte Himalayaberk,Betula utilis 'Doorenbos',2006,14,Gemeentelijk,De Balije,Vleuten - De Meern,1854331,129923.258000,454491.053000,5.021398,52.078127,20200415
8,9,Gewone esdoorn,Acer pseudoplatanus,1969,51,Gemeentelijk,Oog in Al,West,1879612,134689.242000,455388.252000,5.090867,52.086386,20200415
9,10,Zwarte els,Alnus glutinosa 'Laciniata',1993,27,Gemeentelijk,Nieuw Hoograven-zuid,Zuid,1889466,136636.319000,452781.739000,5.119415,52.063027,20200415
